In [1]:
from copy import deepcopy


import networkx as nx

In [2]:
INPUT_TEST_ = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""
INPUT_TEST = [list(p) for p in INPUT_TEST_.split('\n')]

with open('d20_in.txt', 'r') as f:
    INPUT_ = f.read()
INPUT= [list(p) for p in INPUT_.split('\n')]

In [3]:
DIRS = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}

In [4]:
def get_start(map_):
    for i in range(1, len(map_)-1):
        for j in range(1, len(map_[0])-1):
            if map_[i][j] == 'S':
                return (i, j)
    return (-1, -1)

def get_end(map_):
    for i in range(1, len(map_)-1):
        for j in range(1, len(map_[0])-1):
            if map_[i][j] == 'E':
                return (i, j)
    return (-1, -1)

def in_bounds(v, map_):
    i, j = v
    return 1 <= i <= len(map_)-2 and 1 <= j <= len(map_[0])-2

def get_nbhs(v, map_):
    i, j = v
    nbhs = [(i+di, j+dj) for (di, dj) in DIRS.values()]
    return [
        n for n in nbhs
        if in_bounds(n, map_)
        and map_[n[0]][n[1]] != '#'
    ]

def get_extra_nbhs(v, map_):
    i, j = v
    pairs = []
    nbhs_1 = [(i+di, j+dj) 
              for (di, dj) in DIRS.values()
              if in_bounds((i+di, j+dj), map_)
              and map_[i+di][j+dj] == '#']
    for n in nbhs_1:
        ni, nj = n
        pairs += [
            (n, (ni+di, nj+dj))
            for (di, dj) in DIRS.values()
            if (ni+di, nj+dj) != (i, j)
            and map_[ni+di][nj+dj] in 'SE.'
        ]
    return set(pairs)

def print_map(map_):
    print("\n".join(["".join(line) for line in map_]))

In [5]:
BLACK = '\033[30m'
RED = '\033[31m'
GREEN = '\033[32m'
YELLOW = '\033[33m' # orange on some systems
BLUE = '\033[34m'
MAGENTA = '\033[35m'
CYAN = '\033[36m'
LIGHT_GRAY = '\033[37m'
DARK_GRAY = '\033[90m'
BRIGHT_RED = '\033[91m'
BRIGHT_GREEN = '\033[92m'
BRIGHT_YELLOW = '\033[93m'
BRIGHT_BLUE = '\033[94m'
BRIGHT_MAGENTA = '\033[95m'
BRIGHT_CYAN = '\033[96m'
WHITE = '\033[97m'
RESET = '\033[0m' # return to standard terminal text color

def color_map(map_):
    new_map = deepcopy(map_)
    for i in range(len(map_)):
        for j in range(len(map_[0])):
            c = map_[i][j]
            if c == '#':
                new_map[i][j] = DARK_GRAY+c
            elif c in 'SE.':
                new_map[i][j] = BRIGHT_RED+c
            elif c in 'O^v':
                new_map[i][j] = BRIGHT_YELLOW+c
    return new_map

Part 1

In [6]:
map_ = deepcopy(INPUT)

start, end = get_start(map_), get_end(map_)

G = nx.DiGraph()

for i in range(len(map_)):
    for j in range(len(map_[0])):
        if map_[i][j] in 'SE.':
            u = (i, j)
            for v in get_nbhs(u, map_):
                G.add_edge(u, v)

sp = nx.shortest_path(G, source=start, target=end)
max_len = len(sp)-1

print("Shortest path has edge length: ", max_len)

Shortest path has edge length:  9372


In [7]:
cheats_dict = {i: set() for i in range(max_len+1)}

for idx, p in enumerate(sp):
    
    for u, v in get_extra_nbhs(p, map_):
        edges_added = []
        # mutate graph
        if not G.has_edge(p, u):
            G.add_edge(p, u)
            edges_added.append((p, u))
        if not G.has_edge(u, v):
            G.add_edge(u, v)
            edges_added.append((u, v))
        for w in get_nbhs(v, map_):
            if w != u and not G.has_edge(v, w):
                G.add_edge(v, w)
                edges_added.append((v, w))
        # so far we have a new copy of the graph

        sp_new = nx.shortest_path(G, source=start, target=end)
        max_len_new = len(sp_new)-1
        cheats_dict[max_len_new].add((u, v))

        # undo the edges
        for u, v in edges_added:
            G.remove_edge(u, v)

In [8]:
NUM_SAVE = 100

count = 0
for k, v in cheats_dict.items():
    if v and max_len - k >= NUM_SAVE:
        count += len(v)
count

1521

In [ ]:
new_map = deepcopy(map_)
for p in sp:
    i, j = p
    if p not in (start, end):
        new_map[i][j] = 'O'
print_map(color_map(new_map))

Part 2

In [ ]:
map_ = deepcopy(INPUT)
DIST_DUAL = 20
DIST_DIFF = 100

# map_ = deepcopy(INPUT_TEST)
# DIST_DUAL = 20
# DIST_DIFF = 50

start, end = get_start(map_), get_end(map_)

G = nx.DiGraph()

for i in range(len(map_)):
    for j in range(len(map_[0])):
        if map_[i][j] in 'SE.':
            u = (i, j)
            for v in get_nbhs(u, map_):
                G.add_edge(u, v)

sp = nx.shortest_path(G, source=start, target=end)
max_len = len(sp)-1

print("Shortest path has edge length: ", max_len)

Shortest path has edge length:  9372


In [ ]:
count = 0
for i in range(max_len+1):
    start_ = sp[i]
    for j in range(i+1, max_len+1):
    # for j in range(max_len+1):
        end_ = sp[j]
        l1_dist = abs(end_[0]-start_[0]) + abs(end_[1]-start_[1])
        if l1_dist > DIST_DUAL:
            continue
        else:
            dist_saved = j - i - l1_dist
            if dist_saved >= DIST_DIFF:
                count += 1
count